Estoy armando el codigo para ajustar el entrenamiento y verificar que no hay efecto (o cuan grande es) 

Para eso uso el siguiente modelo:

$$Y(i,t) = \alpha_i + \beta \cdot t + \gamma_{inicial} \Theta_{inicial} + \gamma_{final} \Theta_{final}$$

Donde i representa el sujeto, t representa la sesion (aca se juntan una sesion del test, las doce de entrenamiento (separanto los niveles dentro de cada sesion) y el test final (esto habria que revisarlo si conviene o no).  

Como esta regresion lineal es un lio de definir, la idea es hacer un "stack" donde se transforme todo en un sistema de matrices con la forma:

$$ \bar{Y} = \bar{\beta} \cdot \bar{\bar{X}}$$ 

Donde si i va de 1 a 3 y t de 1 a 4 queda escrito de la forma :

$$ X = 
 \begin{pmatrix}
  \beta & \alpha_1 & \alpha_2 & \alpha_3 & \gamma_{inicial} & \gamma_{final} \\
  0 & 1 & 0 & 0 & 1 & 0\\
  1 & 1 & 0 & 0 & 0 & 0\\
  2 & 1 & 0 & 0 & 0 & 0\\
  3 & 1 & 0 & 0 & 0 & 1\\
  0 & 0 & 1 & 0 & 1 & 0\\
  1 & 0 & 1 & 0 & 0 & 0\\
  2 & 0 & 1 & 0 & 0 & 0\\
  3 & 0 & 1 & 0 & 0 & 1\\
  0 & 0 & 0 & 1 & 1 & 0\\
  1 & 0 & 0 & 1 & 0 & 0\\
  2 & 0 & 0 & 1 & 0 & 0\\
  3 & 0 & 0 & 1 & 0 & 1\\
 \end{pmatrix}
$$

Una vez generada esa matriz usamos el codigo que figura aca:

http://stackoverflow.com/questions/24393518/python-sklearn-linear-model-linearregression-working-weird

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from scripts.db import *
from scripts.plots import *
from scripts.stats import *
from scripts.statsTransferencia import *
import numpy as np
import statsmodels.api as sm

Aca hacemos el chequeo de si hay aprendizaje o no desde el principio hasta el final (sin contar test final)

In [8]:
# Armamos la matriz

sujetos = 4
secuencias = 12 + 1

betas = [i for i in range(secuencias)] * sujetos + [0,1,2,3]

alphas = [[]*(sujetos+1)]*(sujetos+1)

for sujetox in range(sujetos+1):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * secuencias
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * secuencias

for sujetox in range(sujetos):
    alphas[sujetox] = alphas[sujetox] + [0] * 4
    
alphas[sujetos] = alphas[sujetos] + [1] * 5
        
gamas = ([1] + [0] * (secuencias - 1))*sujetos + [1,0,0,0]

matriz = [betas] + alphas + [gamas]

matriz = list(map(list, zip(*matriz)))

# Tomamos los datos de la base de datos
resumen = dataNumerica()
valores = []
nombres = []
for alias, contenido in resumen.items():
    performance = []
    if contenido['Orientacion'] != 'CONTROL':
        
        if sum (contenido['EntrenamientoInicialEnAngulos']) > 200:
            if contenido['Orientacion'] == 'P30':
                performance = performance + [contenido['TestInicialEnAngulos'][0]]
            if contenido['Orientacion'] == 'A30':
                performance = performance + [contenido['TestInicialEnAngulos'][4]]
            for i in range(4):    
                performance = performance + [contenido['EntrenamientoInicialEnAngulos'][i]]
                performance = performance + [contenido['EntrenamientoMedioEnAngulos'][i]]
                performance = performance + [contenido['EntrenamientoFinal'][i]]
            valores = valores + performance
            nombres = nombres +[alias]
        else:
            cola = []
            if contenido['Orientacion'] == 'P30':
                cola = cola + [contenido['TestInicial'][0]]
            if contenido['Orientacion'] == 'A30':
                cola = cola + [contenido['TestInicial'][4]]
            for i in range(1):
                cola = cola + [contenido['EntrenamientoInicial'][i]]
                cola = cola + [contenido['EntrenamientoMedio'][i]]
                cola = cola + [contenido['EntrenamientoFinal'][i]]
valores = valores + cola
nombres = nombres +['Ivan']

x = np.array(matriz)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary())

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:348: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:320: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     9.748
Date:                Mon, 31 Oct 2016   Prob (F-statistic):           4.68e-07
Time:                        16:36:08   Log-Likelihood:                -258.53
No. Observations:                  56   AIC:                             531.1
Df Residuals:                      49   BIC:                             545.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1             2.4756      1.080      2.292      0.0

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:322: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


Hacemos el ajuste considerando tambien el test final

In [10]:
# Armamos la matriz
# En todos lados hay correcciones para uncluir al sujeto que hizo solo un entrenamiento

sujetos = 4
secuencias = 12 + 1 + 1

betas = [i for i in range(secuencias)] * sujetos + [0,1,2,3,4]

alphas = [[]*(sujetos+1)]*(sujetos+1)

for sujetox in range(sujetos+1):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * secuencias
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * secuencias

for sujetox in range(sujetos):
    alphas[sujetox] = alphas[sujetox] + [0] * 5
    
alphas[sujetos] = alphas[sujetos] + [1] * 5
        
gamas1 = ([1] + [0] * (secuencias - 1))*sujetos + [1,0,0,0,0]
gamas2 = ([0]* (secuencias - 1) + [1])*sujetos + [0,0,0,0,1]


matriz = [betas] + alphas + [gamas1] + [gamas2]

matriz = list(map(list, zip(*matriz)))

# Tomamos los datos de la base de datos
resumen = dataNumerica()
valores = []
nombres = []
for alias, contenido in resumen.items():
    performance = []
    if contenido['Orientacion'] != 'CONTROL':
        
        if sum (contenido['EntrenamientoInicial']) > 200: # Filtramos al usuario con menos datos
            # Levantamos el valor del test inicial
            if contenido['Orientacion'] == 'P30':
                performance = performance + [contenido['TestInicial'][0]]
            if contenido['Orientacion'] == 'A30':
                performance = performance + [contenido['TestInicial'][4]]
            # Levantamos los valores de entrenamiento
            for i in range(4):    
                performance = performance + [contenido['EntrenamientoInicial'][i]]
                performance = performance + [contenido['EntrenamientoMedio'][i]]
                performance = performance + [contenido['EntrenamientoFinal'][i]]
            # Levantamos el valor del test final
            if contenido['Orientacion'] == 'P30':
                performance = performance + [contenido['TestFinal'][0]]
            if contenido['Orientacion'] == 'A30':
                performance = performance + [contenido['TestFinal'][4]]
            # Agregamos los valores al vector de respuestas
            valores = valores + performance
            nombres = nombres + [alias]
        else:
            cola = []
            aliascola = [alias]
            if contenido['Orientacion'] == 'P30':
                cola = cola + [contenido['TestInicial'][0]]
            if contenido['Orientacion'] == 'A30':
                cola = cola + [contenido['TestInicial'][4]]
            for i in range(1):
                cola = cola + [contenido['EntrenamientoInicial'][i]]
                cola = cola + [contenido['EntrenamientoMedio'][i]]
                cola = cola + [contenido['EntrenamientoFinal'][i]]
            if contenido['Orientacion'] == 'P30':
                cola = cola + [contenido['TestFinal'][0]]
            if contenido['Orientacion'] == 'A30':
                cola = cola + [contenido['TestFinal'][4]]
                
valores = valores + cola
nombres = nombres + aliascola

x = np.array(matriz)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary())

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:348: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:320: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     9.068
Date:                Mon, 31 Oct 2016   Prob (F-statistic):           2.61e-07
Time:                        17:11:46   Log-Likelihood:                -279.94
No. Observations:                  61   AIC:                             575.9
Df Residuals:                      53   BIC:                             592.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1             2.6804      1.037      2.584      0.0

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:322: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


Aca hacemos el mismo analisis incluyendo test inicial y final pero sacando al sujeto que tiene un marcado aprendizaje en la sesion uno

In [19]:
# Armamos la matriz
# En todos lados hay correcciones para uncluir al sujeto que hizo solo un entrenamiento
# Sacamos al sujeto que mejora en la primer sesion

sujetos = 3
secuencias = 12 + 1 + 1

betas = [i for i in range(secuencias)] * sujetos + [0,1,2,3,4]

alphas = [[]*(sujetos+1)]*(sujetos+1)

for sujetox in range(sujetos+1):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * secuencias
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * secuencias

for sujetox in range(sujetos):
    alphas[sujetox] = alphas[sujetox] + [0] * 5
    
alphas[sujetos] = alphas[sujetos] + [1] * 5
        
gamas1 = ([1] + [0] * (secuencias - 1))*sujetos + [1,0,0,0,0]
gamas2 = ([0]* (secuencias - 1) + [1])*sujetos + [0,0,0,0,1]


matriz = [betas] + alphas + [gamas1] + [gamas2]

matriz = list(map(list, zip(*matriz)))

# Tomamos los datos de la base de datos
resumen = dataNumerica()
valores = []
nombres = []
for alias, contenido in resumen.items():
    performance = []
    if alias != 'user01': 

        if contenido['Orientacion'] != 'CONTROL':

            if sum (contenido['EntrenamientoInicial']) > 200: # Filtramos al usuario con menos datos
                # Levantamos el valor del test inicial
                if contenido['Orientacion'] == 'P30':
                    performance = performance + [contenido['TestInicial'][0]]
                if contenido['Orientacion'] == 'A30':
                    performance = performance + [contenido['TestInicial'][4]]
                # Levantamos los valores de entrenamiento
                for i in range(4):    
                    performance = performance + [contenido['EntrenamientoInicial'][i]]
                    performance = performance + [contenido['EntrenamientoMedio'][i]]
                    performance = performance + [contenido['EntrenamientoFinal'][i]]
                # Levantamos el valor del test final
                if contenido['Orientacion'] == 'P30':
                    performance = performance + [contenido['TestFinal'][0]]
                if contenido['Orientacion'] == 'A30':
                    performance = performance + [contenido['TestFinal'][4]]
                # Agregamos los valores al vector de respuestas
                valores = valores + performance
                nombres = nombres + [alias]
            else:
                cola = []
                aliascola = [alias]
                if contenido['Orientacion'] == 'P30':
                    cola = cola + [contenido['TestInicial'][0]]
                if contenido['Orientacion'] == 'A30':
                    cola = cola + [contenido['TestInicial'][4]]
                for i in range(1):
                    cola = cola + [contenido['EntrenamientoInicial'][i]]
                    cola = cola + [contenido['EntrenamientoMedio'][i]]
                    cola = cola + [contenido['EntrenamientoFinal'][i]]
                if contenido['Orientacion'] == 'P30':
                    cola = cola + [contenido['TestFinal'][0]]
                if contenido['Orientacion'] == 'A30':
                    cola = cola + [contenido['TestFinal'][4]]
                
valores = valores + cola
nombres = nombres + aliascola

print (nombres)
x = np.array(matriz)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary())

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:348: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:320: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)


['user03', 'user02', 'user05', 'user04']
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.465
Method:                 Least Squares   F-statistic:                     7.653
Date:                Mon, 31 Oct 2016   Prob (F-statistic):           1.64e-05
Time:                        17:35:17   Log-Likelihood:                -202.08
No. Observations:                  47   AIC:                             418.2
Df Residuals:                      40   BIC:                             431.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1         

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:322: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


Aca hacemos el chequeo de si hay aprendizaje en los ultimos 3 sesiones de entrenamiento (esto no incluye al sujeto que no hizo todo el entrenamiendo)

In [12]:
# Armamos la matriz

sujetos = 4
secuencias = 9

betas = [i for i in range(secuencias)] * sujetos

alphas = [[]*(sujetos)]*(sujetos)

for sujetox in range(sujetos):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * secuencias
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * secuencias

matriz = [betas] + alphas

matriz = list(map(list, zip(*matriz)))

# Tomamos los datos de la base de datos
resumen = dataNumerica()
valores = []
nombres = []
for alias, contenido in resumen.items():
    performance = []
    if contenido['Orientacion'] != 'CONTROL':
        
        if sum (contenido['EntrenamientoInicial']) > 200:
            for i in range(3):    
                performance = performance + [contenido['EntrenamientoInicial'][i+1]]
                performance = performance + [contenido['EntrenamientoMedio'][i+1]]
                performance = performance + [contenido['EntrenamientoFinal'][i+1]]
            valores = valores + performance
            nombres = nombres + [alias]

valores = valores
nombres = nombres

x = np.array(matriz)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary().as_latex())

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:348: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:320: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)


\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &        y         & \textbf{  R-squared:         } &     0.356   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.273   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     4.292   \\
\textbf{Date:}             & Mon, 31 Oct 2016 & \textbf{  Prob (F-statistic):} &  0.00704    \\
\textbf{Time:}             &     17:12:46     & \textbf{  Log-Likelihood:    } &   -132.13   \\
\textbf{No. Observations:} &          36      & \textbf{  AIC:               } &     274.3   \\
\textbf{Df Residuals:}     &          31      & \textbf{  BIC:               } &     282.2   \\
\textbf{Df Model:}         &           4      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lccccc}
            & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$>$$|$t$|$} & \textbf{[95.0\% Conf. Int.]}  \\
\midrule
\textbf{x

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:322: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


Hacemos un analisis separando de entrenamiento paralelismo y entrenamiento angulos para comparar el salto en el primer entrenamiento

In [27]:
# Armamos la matriz

sujetos = 4
secuencias = 12 + 1

betas = [i for i in range(secuencias)] * sujetos + [0,1,2,3]

alphas = [[]*(sujetos+1)]*(sujetos+1)

for sujetox in range(sujetos+1):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * secuencias
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * secuencias

for sujetox in range(sujetos):
    alphas[sujetox] = alphas[sujetox] + [0] * 4
    
alphas[sujetos] = alphas[sujetos] + [1] * 5
        
# Tomamos los datos de la base de datos
resumen = dataNumerica()
valores = []
nombres = []
for alias, contenido in resumen.items():
    performance = []
    if contenido['Orientacion'] != 'CONTROL':
        
        if sum (contenido['EntrenamientoInicial']) > 200:
            if contenido['Orientacion'] == 'P30':
                performance = performance + [contenido['TestInicial'][0]]
            if contenido['Orientacion'] == 'A30':
                performance = performance + [contenido['TestInicial'][4]]
            for i in range(4):    
                performance = performance + [contenido['EntrenamientoInicial'][i]]
                performance = performance + [contenido['EntrenamientoMedio'][i]]
                performance = performance + [contenido['EntrenamientoFinal'][i]]
            valores = valores + performance
            nombres = nombres +[alias]
        else:
            cola = []
            if contenido['Orientacion'] == 'P30':
                cola = cola + [contenido['TestInicial'][0]]
            if contenido['Orientacion'] == 'A30':
                cola = cola + [contenido['TestInicial'][4]]
            for i in range(1):
                cola = cola + [contenido['EntrenamientoInicial'][i]]
                cola = cola + [contenido['EntrenamientoMedio'][i]]
                cola = cola + [contenido['EntrenamientoFinal'][i]]

gamas = [[],[]]

for nombre in nombres:
    print (nombre)
    if nombre in ['ExpT_Magdalena', 'ExpT_Enzo']:
        gamas[0] = gamas[0] + ([1] + [0] * (secuencias - 1))
        gamas[1] = gamas[1] + ([0] + [0] * (secuencias - 1))
    else:
        gamas[0] = gamas[0] + ([0] + [0] * (secuencias - 1))
        gamas[1] = gamas[1] + ([1] + [0] * (secuencias - 1))

gamas[0] = gamas[0] + [1,0,0,0]
gamas[1] = gamas[1] + [0,0,0,0]

matriz = [betas] + alphas + gamas

matriz = list(map(list, zip(*matriz)))

valores = valores + cola
nombres = nombres +['Ivan']

x = np.array(matriz)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary())

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:297: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:269: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)


ExpT_Magdalena
ExpT_Carolina
ExpT_Enzo
ExpT_Marco
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     20.85
Date:                Thu, 27 Oct 2016   Prob (F-statistic):           1.43e-12
Time:                        14:01:58   Log-Likelihood:                -241.43
No. Observations:                  56   AIC:                             498.9
Df Residuals:                      48   BIC:                             515.1
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:271: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


Hacemos solo paralelismo

In [ ]:
# Armamos la matriz

sujetos = 2
secuencias = 12 + 1

betas = [i for i in range(secuencias)] * sujetos + [0,1,2,3]

alphas = [[]*(sujetos+1)]*(sujetos+1)

for sujetox in range(sujetos+1):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * secuencias
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * secuencias

for sujetox in range(sujetos):
    alphas[sujetox] = alphas[sujetox] + [0] * 4
    
alphas[sujetos] = alphas[sujetos] + [1] * 5
        
# Tomamos los datos de la base de datos
resumen = dataNumerica()
valores = []
nombres = []
for alias, contenido in resumen.items():
    performance = []
    if contenido['Orientacion'] == 'P30':
        if sum (contenido['EntrenamientoInicial']) > 200:
            performance = performance + [contenido['TestInicial'][0]]
            for i in range(4):    
                performance = performance + [contenido['EntrenamientoInicial'][i]]
                performance = performance + [contenido['EntrenamientoMedio'][i]]
                performance = performance + [contenido['EntrenamientoFinal'][i]]
            valores = valores + performance
            nombres = nombres +[alias]
        else:
            cola = []
            cola = cola + [contenido['TestInicial'][0]]
            for i in range(1):
                cola = cola + [contenido['EntrenamientoInicial'][i]]
                cola = cola + [contenido['EntrenamientoMedio'][i]]
                cola = cola + [contenido['EntrenamientoFinal'][i]]

gamas = [[],[]]

for nombre in nombres:
    print (nombre)
    if nombre in ['ExpT_Magdalena', 'ExpT_Enzo']:
        gamas[0] = gamas[0] + ([1] + [0] * (secuencias - 1))
        gamas[1] = gamas[1] + ([0] + [0] * (secuencias - 1))
    else:
        gamas[0] = gamas[0] + ([0] + [0] * (secuencias - 1))
        gamas[1] = gamas[1] + ([1] + [0] * (secuencias - 1))

gamas[0] = gamas[0] + [1,0,0,0]
gamas[1] = gamas[1] + [0,0,0,0]

matriz = [betas] + alphas + gamas

matriz = list(map(list, zip(*matriz)))

valores = valores + cola
nombres = nombres +['Ivan']

x = np.array(matriz)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary())

Hacemos el analisis de la evolucion intra sesion. Primero considerando solo la variable temporal. Despues agregaqndo un factor fijo usuario. Despues un factor fijo por dia. Y por ultimo por ambas variables (que es equivalente a restar la media a cada sesion)

In [69]:
# Armamos la matriz, para descartar los tres datos que queremos tirar, armamos la matriz en gral y despues borramos esas tres filas
# De hecho es puede hacer eso en gral.

sujetos = 5
dias = 4
secuencias = 3
sesiones = dias * sujetos
niveles = dias * secuencias

betas = [i for i in range(secuencias)] * sesiones + [0,1,2]

alphas = [[]*(sujetos)]*(sujetos)

for sujetox in range(sujetos):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * niveles
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * niveles

detas = [[]*dias]*dias

for sujeto in range(sujetos):
    for diax in range (dias):
        for diay in range (dias):
            if diax==diay:
                detas[diax] = detas[diax] + [1] * secuencias
            else:
                detas[diax] = detas[diax] + [0] * secuencias

feedback = [0,1,0] * dias * sujetos

# Tomamos los datos de la base de datos
resumen = dataNumerica()
valores = []
nombres = []
for alias, contenido in resumen.items():
    if contenido['Orientacion'] != 'CONTROL':
        for i in range(dias):
            valores = valores + [contenido['EntrenamientoInicial'][i]]
            valores = valores + [contenido['EntrenamientoMedio'][i]]
            valores = valores + [contenido['EntrenamientoFinal'][i]]
        nombres = nombres +[alias]

# Marcamos las ternas para eliminar con ceros
for i in range(len(valores)):
    if valores[i] != 0:
        if valores[i] < 50:
            valores[i] = 0
            valores[i+1] = 0
            valores[i+2] = 0

# Aca eligiendo que termino sumar y cual no tengo los diferentes analisis
# matriz = [[1] * len(betas)] + [betas] + alphas + detas + [feedback] + [valores]
matriz = [[1] * len(betas)] + [betas] + [feedback] + [valores]
          
matriz = list(map(list, zip(*matriz)))

matriz = [item for item in matriz if item[-1]]

#print (np.matrix(matriz))
matrizPura = [item[:-1] for item in matriz]
valores = [item[-1] for item in matriz]


x = np.array(matrizPura)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary())

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:348: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:320: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.561
Date:                Wed, 02 Nov 2016   Prob (F-statistic):              0.221
Time:                        14:05:32   Log-Likelihood:                -182.31
No. Observations:                  48   AIC:                             370.6
Df Residuals:                      45   BIC:                             376.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        161.6875      2.788     57.999      0.0

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:322: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)


In [71]:
# Armamos la matriz, para descartar los tres datos que queremos tirar, armamos la matriz en gral y despues borramos esas tres filas
# De hecho es puede hacer eso en gral.

sujetos = 5
dias = 4
secuencias = 3
sesiones = dias * sujetos
niveles = dias * secuencias

betas = [i for i in range(secuencias)] * sesiones + [0,1,2]

alphas = [[]*(sujetos)]*(sujetos)

for sujetox in range(sujetos):
    for sujetoy in range(sujetos):
        if sujetox==sujetoy:
            alphas[sujetox] = alphas[sujetox] + [1] * niveles
        else:
            alphas[sujetox] = alphas[sujetox] + [0] * niveles

detas = [[]*dias]*dias

for sujeto in range(sujetos):
    for diax in range (dias):
        for diay in range (dias):
            if diax==diay:
                detas[diax] = detas[diax] + [1] * secuencias
            else:
                detas[diax] = detas[diax] + [0] * secuencias

feedback = [0,1,0] * dias * sujetos

# Tomamos los datos de la base de datos
resumen = dataNumerica()
valores = []
nombres = []
for alias, contenido in resumen.items():
    if contenido['Orientacion'] != 'CONTROL':
        for i in range(dias):
            valores = valores + [contenido['EntrenamientoInicial'][i]]
            valores = valores + [contenido['EntrenamientoMedio'][i]]
            valores = valores + [contenido['EntrenamientoFinal'][i]]
        nombres = nombres +[alias]

# Marcamos las ternas para eliminar con ceros
for i in range(len(valores)):
    if valores[i] != 0:
        if valores[i] < 50:
            valores[i] = 0
            valores[i+1] = 0
            valores[i+2] = 0

for i in range(len(valores)):
    if i%3 == 1:
        valores[i] = 0

print (valores)

# Aca eligiendo que termino sumar y cual no tengo los diferentes analisis
# matriz = [[1] * len(betas)] + [betas] + alphas + detas + [feedback] + [valores]
matriz = [[1] * len(betas)] + [betas] + [valores]
          
matriz = list(map(list, zip(*matriz)))

matriz = [item for item in matriz if item[-1]]

#print (np.matrix(matriz))
matrizPura = [item[:-1] for item in matriz]
valores = [item[-1] for item in matriz]


x = np.array(matrizPura)  
y = np.array(valores)

mod = sm.OLS(y, x)
res = mod.fit()
print (res.summary())

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:348: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_FaseActiva].replace('ExperimentoCompleto', 'TestFinal', inplace=True)
/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:320: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('A30', 'P30', inplace=True)


[165, 0, 183, 0, 0, 0, 0, 0, 0, 0, 0, 0, 157, 0, 162, 132, 0, 174, 154, 0, 164, 163, 0, 180, 150, 0, 159, 158, 0, 166, 166, 0, 152, 151, 0, 159, 152, 0, 167, 169, 0, 175, 174, 0, 185, 180, 0, 179, 0, 0, 0, 186, 0, 164, 170, 0, 164, 160, 0, 152]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     2.153
Date:                Wed, 02 Nov 2016   Prob (F-statistic):              0.153
Time:                        14:14:33   Log-Likelihood:                -123.37
No. Observations:                  32   AIC:                             250.7
Df Residuals:                      30   BIC:                             253.7
Df Model:                           1                                         
Covariance Type:            nonrobust       

/home/ionatan/Gits/IPyVisound/scripts/statsTransferencia.py:322: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame
  dataByAlias[cts.P_OrientacionEntrenamiento].replace('P30', 'CONTROL', inplace=True)
